In [1]:
import os

In [2]:
# Go to the src directory
os.chdir("C:\\Users\\Ronan\\Documents\\ML\\Image_Classification\\Picture_Classification_ML_Project\\src")

In [3]:
# Check the current working directory
%pwd

'C:\\Users\\Ronan\\Documents\\ML\\Image_Classification\\Picture_Classification_ML_Project\\src'

In [4]:
# Modify the return types of the config.yaml file
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from CNN_Classification_Task.constants import *
from CNN_Classification_Task.utils.common import read_yaml, create_directories

C:\Users\Ronan\Documents\ML\Image_Classification\Picture_Classification_ML_Project\src\CNN_Classification_Task\constants\__init__.py:6: SyntaxWarning: invalid escape sequence '\P'
  PARAMS_FILE_PATH = Path("C:/Users/Ronan/Documents/ML/Image_Classification\Picture_Classification_ML_Project/params.yaml")


In [6]:
class ConfigurationManager:
    # Read the config.yaml file and the params.yaml file
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # Setup the configuration of the data ingestion task
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [7]:
import os
import urllib.request as request
import zipfile
from CNN_Classification_Task import logger
from CNN_Classification_Task.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # Downloads the file from the source URL and saves it to the local_data_file
    def download_file(self):
        # Check if the file already exists and download it
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            # Logs if the download was successful
            logger.info(f"{filename} is downloaded with the following infos: \n{headers}")
        else:
            # Logs if the file already exists
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    # Extracts the zip file into the data directory
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        # Check if the directory already exists and create it if not
        os.makedirs(unzip_path, exist_ok=True)
        # Extracts the zip file into the data directory
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [9]:
# Instantiate the DataIngestion class
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
# If an exception is raised, print the error message    
except Exception as e:
    raise e

[2024-05-18 16:07:58,994: INFO: common: yaml file: C:\Users\Ronan\Documents\ML\Image_Classification\Picture_Classification_ML_Project\config\config.yaml loaded successfully]
[2024-05-18 16:07:58,994: INFO: common: yaml file: C:\Users\Ronan\Documents\ML\Image_Classification\Picture_Classification_ML_Project\params.yaml loaded successfully]
[2024-05-18 16:07:58,996: INFO: common: created directory at: artifacts]
[2024-05-18 16:07:58,997: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-18 16:07:59,966: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-Git